In [1]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[2]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
import minsearch

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [87]:
import os
from google import genai
os.environ["GOOGLE_API_KEY"] = "YOUR KEY HERE"

In [10]:
client = genai.Client()

In [11]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [12]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [83]:
def llm(prompt):
    response = client.models.generate_content(
    model="gemini-2.0-flash", 
    contents= prompt 
    )
    return(response.text)


In [14]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [15]:
rag('how do I run kafka?')

'AI learns patterns from data to make predictions or decisions.\n'

In [16]:
rag('the course has already started, can I still enroll?')

'AI learns patterns from data to make predictions or decisions.\n'

In [17]:
from qdrant_client import QdrantClient, models

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
qd_client = QdrantClient("http://localhost:6333")

In [19]:
EMBEDDING_DIMENSIONALITY = 512
model_handle = "jinaai/jina-embeddings-v2-small-en"

In [30]:
#collection_name = "zoomcamp-faq"
collection_name

'zoomcamp-faq'

In [28]:
qd_client.delete_collection(collection_name=collection_name)

True

In [29]:
qd_client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [36]:
points = []

for i, doc in enumerate(documents):
    text =  doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text,model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector, #models.Document(text=doc['text'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload= doc
    )
    points.append(point)


In [38]:
points[1]

PointStruct(id=1, vector=Document(text='Course - What are the prerequisites for this course? GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites', model='jinaai/jina-embeddings-v2-small-en', options=None), payload={'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites', 'section': 'General course-related questions', 'question': 'Course - What are the prerequisites for this course?', 'course': 'data-engineering-zoomcamp'})

In [40]:
qd_client.upsert(
    collection_name=collection_name,
    points=points
)

Fetching 5 files: 100%|██████████████████████████████████████████████████████████████████████| 5/5 [00:02<00:00,  1.80it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [45]:
question = ' I just discovered the course. Can I still enroll?'

In [47]:
query_points = qd_client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=question,
        model=model_handle 
    ),
    limit=5, # top closest matches
    with_payload=True 
)


In [55]:
query_points.points[0]

ScoredPoint(id=2, version=0, score=0.8561493, payload={'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}, vector=None, shard_key=None, order_value=None)

In [51]:
results =[]
for point in query_points.points:
    results.append(point.payload)

In [56]:
results[0]

{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
 'section': 'General course-related questions',
 'question': 'Course - Can I still join the course after the start date?',
 'course': 'data-engineering-zoomcamp'}

In [58]:
qd_client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword" # exact matching on string metadata fields
)


UpdateResult(operation_id=2, status=<UpdateStatus.COMPLETED: 'completed'>)

In [59]:
course = 'data-engineering-zoompcamp'
query_points = qd_client.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=question,
        model=model_handle 
    ),
    query_filter=models.Filter( # filter by course name
        must=[
            models.FieldCondition(
                key="course",
                match=models.MatchValue(value=course)
            )
        ]
    ),
    limit=5, # top closest matches
    with_payload=True 
)


In [78]:
def vector_search(question):
    
    course = 'data-engineering-zoomcamp'
    query_points = qd_client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model=model_handle 
        ),
        query_filter=models.Filter( # filter by course name
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=5, # top closest matches
        with_payload=True 
    )
    
    results = []
        
    for point in query_points.points:
        results.append(point.payload)
    
    return results

In [79]:
vector_search('how do i run kafka')

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'For example, when running JsonConsumer.java, got:\nConsuming form kafka started\nRESULTS:::0\nRESULTS:::0\nRESULTS:::0\nOr when running JsonProducer.java, got:\nException in thread "main" java.util.concurrent.ExecutionException: org.apache.kafka.common.errors.SaslAuthenticationException: Authentication failed\nSolution:\nMake sure in the scripts in src/main/java/org/example/ that you are running (e.g. JsonConsumer.java, JsonProducer.java), the StreamsConfig.BOOTSTRAP_SERVERS_CONFIG is the correct server url (e.g. europe-west3 from example vs europe-west2)\nMake sure cluster key and secrets are updated in src/main/java/org/example/Secrets.java (KAFKA_CLUSTER_KEY and KA

In [84]:
def rag(query):
    search_results = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [85]:
rag('how do i enroll in course')

'You can start learning and submitting homework without registering. Registration is just to gauge interest before the start date.\n'

In [86]:
rag('how do solve kafka error')

'Here are some possible solutions to Kafka errors:\n\n*   **`kafka.errors.NoBrokersAvailable: NoBrokersAvailable`**: This usually means your Kafka broker Docker container isn\'t running. Use `docker ps` to check, and if needed, run `docker compose up -d` in the Docker Compose YAML file folder to start the instances.\n*   **Java Kafka - No results retrieved or no message sent**: Ensure that the `StreamsConfig.BOOTSTRAP_SERVERS_CONFIG` in your Java scripts (e.g., `JsonConsumer.java`, `JsonProducer.java`) has the correct server URL. Also, verify that the `KAFKA_CLUSTER_KEY` and `KAFKA_CLUSTER_SECRET` in `src/main/java/org/example/Secrets.java` are up-to-date.\n*   **`ModuleNotFoundError: No module named \'kafka.vendor.six.moves\'`**: Use `pip install kafka-python-ng` instead of `pip install kafka-python`.\n*   **Running stream-kafka script gets stuck on a loop with Connection Refused**: Check the logs of the `message_queue` container in Docker. If it\'s restarting with an "insufficient ph